# functions: write_results

[write_results](../api/functions.rst#nornir_utils.plugins.functions.write_results) is a function that writes result to files with hostname names. This function is based on [print_result](../api/functions.rst#nornir_utils.plugins.functions.print_result) function with `file` argument:

In [1]:
from nornir import InitNornir

def dummy_task(task):
    return "hi!!!"

nr = InitNornir(
    inventory={
        "plugin":"YAMLInventory",
        "options": {
            "host_file": "data/hosts.yaml",
            "group_file": "data/groups.yaml",
            "defaults_file": "data/defaults.yaml",
        }
    }
)

result = nr.run(task=dummy_task)

Now we could write the results easily with the `write_results` function:

In [2]:
from nornir_utils.plugins.functions import write_results

write_results(result, dirpath="out_files/write_results")

```text
$ ls out_files/write_results

dev1.group_1  dev2.group_1  dev3.group_2  dev4.group_2	dev5.no_group
```

Let's look at the content of each file:

In [3]:
import os

dir_path = "out_files/write_results/"

for filename in sorted(os.listdir(dir_path)):
    file_path = dir_path + filename
   
    with open(file_path) as f:
        print("\033[1m" + file_path + "\033[0;0m", f.read(), sep='\n')

out_files/write_results/dev1.group_1
dummy_task**********************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv dummy_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
hi!!!
^^^^ END dummy_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

out_files/write_results/dev2.group_1
dummy_task**********************************************************************
* dev2.group_1 ** changed : False **********************************************
vvvv dummy_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
hi!!!
^^^^ END dummy_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

out_files/write_results/dev3.group_2
dummy_task**********************************************************************
* dev3.group_2 ** changed : False **********************************************
vvvv dummy_task ** changed : False vvvvvvvvvvvvvvvvvvvv

`dirpath` here is path to directory, you want to write the results for each host. Any missing directories from the `dirpath` argument are created as needed.

## Writing specific data

If the task returns different information, you can also select which ones to write. For instance:

In [4]:
from nornir.core.task import Result

def task_with_data(task):
    return Result(host=task.host, result="Spaghetti monster", sw_char="Jar Jar Binks", food="hawaiian pizza", OS="windows")


fav_result = nr.run(task=task_with_data)

# write only sw_char and food vars
write_results(fav_result, dirpath="out_files/write_results", vars=["sw_char", "food"])

In [5]:
for filename in sorted(os.listdir(dir_path)):
    file_path = dir_path + filename
   
    with open(file_path) as f:
        print("\033[1m" + file_path + "\033[0;0m", f.read(), sep='\n')

out_files/write_results/dev1.group_1
task_with_data******************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv task_with_data ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
Jar Jar Binks
hawaiian pizza
^^^^ END task_with_data ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

out_files/write_results/dev2.group_1
task_with_data******************************************************************
* dev2.group_1 ** changed : False **********************************************
vvvv task_with_data ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
Jar Jar Binks
hawaiian pizza
^^^^ END task_with_data ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

out_files/write_results/dev3.group_2
task_with_data******************************************************************
* dev3.group_2 ** changed : False **********************************************
vvvv task

## Severity

You can also ask `write_results` to write the results only if the severity is equal or superior to the one specified. This is particularly useful if your script is very large and you don't care about intermediate results. For instance:

In [6]:
import logging

def complex_task(task):
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    return "I did a lot of things!!!"

complex_result = nr.run(task=complex_task)
complex_result

AggregatedResult (complex_task): {'dev1.group_1': MultiResult: [Result: "complex_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task"], 'dev2.group_1': MultiResult: [Result: "complex_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task"], 'dev3.group_2': MultiResult: [Result: "complex_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task"], 'dev4.group_2': MultiResult: [Result: "complex_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task"], 'dev5.no_group': MultiResult: [Result: "complex_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task", Result: "dummy_task"]}

`write_results` will only write `INFO` severity and above by default, so, if we write the result, we should only see the result of the parent task:

In [7]:
write_results(complex_result, dirpath="out_files/write_results")

In [8]:
for filename in sorted(os.listdir(dir_path)):
    file_path = dir_path + filename
   
    with open(file_path) as f:
        print("\033[1m" + file_path + "\033[0;0m", f.read(), sep='\n')

out_files/write_results/dev1.group_1
complex_task********************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
^^^^ END complex_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

out_files/write_results/dev2.group_1
complex_task********************************************************************
* dev2.group_1 ** changed : False **********************************************
vvvv complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
^^^^ END complex_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

out_files/write_results/dev3.group_2
complex_task********************************************************************
* dev3.group_2 ** changed : False **********************************************
vvvv complex_task

If you want to write all the tasks for debugging purposes you could specify it with:

In [9]:
write_results(complex_result, dirpath="out_files/write_results", severity_level=logging.DEBUG)

In [10]:
for filename in sorted(os.listdir(dir_path)):
    file_path = dir_path + filename
   
    with open(file_path) as f:
        print("\033[1m" + file_path + "\033[0;0m", f.read(), sep='\n')

out_files/write_results/dev1.group_1
complex_task********************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
^^^^ END complex_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

out_files/write_results/dev2.group_1
complex_task********************************************************************
* dev2.group_1 ** changed

## No_errors

If some `Result` objects have exceptions, then you can exclude them from the files:

In [11]:
def task_with_exception(task):
    return Result(host=task.host, result="Something went wrong", exception=Exception())


def task_without_exception(task):
    return "All is fine"

def task(task):
    task.run(task=task_with_exception)
    task.run(task=task_without_exception)
    return "What's happening?"

    
result_with_exception = nr.run(task=task)

write_results(result_with_exception, dirpath="out_files/write_results")

In [12]:
for filename in sorted(os.listdir(dir_path)):
    file_path = dir_path + filename
   
    with open(file_path) as f:
        print("\033[1m" + file_path + "\033[0;0m", f.read(), sep='\n')

out_files/write_results/dev1.group_1
task****************************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
What's happening?
---- task_with_exception ** changed : False ------------------------------------ INFO
Something went wrong
---- task_without_exception ** changed : False --------------------------------- INFO
All is fine
^^^^ END task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

out_files/write_results/dev2.group_1
task****************************************************************************
* dev2.group_1 ** changed : False **********************************************
vvvv task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
What's happening?
---- task_with_exception ** changed : False ------------------------------------ INFO
Something went wrong


In [13]:
write_results(result_with_exception, dirpath="out_files/write_results", no_errors=True)

In [14]:
for filename in sorted(os.listdir(dir_path)):
    file_path = dir_path + filename
   
    with open(file_path) as f:
        print("\033[1m" + file_path + "\033[0;0m", f.read(), sep='\n')

out_files/write_results/dev1.group_1
task****************************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
What's happening?
---- task_without_exception ** changed : False --------------------------------- INFO
All is fine
^^^^ END task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

out_files/write_results/dev2.group_1
task****************************************************************************
* dev2.group_1 ** changed : False **********************************************
vvvv task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
What's happening?
---- task_without_exception ** changed : False --------------------------------- INFO
All is fine
^^^^ END task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

out_files/write_results/dev3.group

## Write_host

`write_host` argument can help you, if you don't need to write hostnames to files for `MultiResult` or `Result` object. Since `write_results` writes the results to files with hostname names, this can be useful.

For `MultiResult`:

In [15]:
write_results(result['dev1.group_1'], dirpath="out_files/write_results", write_host=False)

In [16]:
with open("out_files/write_results/dev1.group_1") as f:
    print(f.read())

vvvv dummy_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
hi!!!
^^^^ END dummy_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^



For `Result`:

In [17]:
write_results(result['dev1.group_1'][0], dirpath="out_files/write_results", write_host=False)

In [18]:
with open("out_files/write_results/dev1.group_1") as f:
    print(f.read())

---- dummy_task ** changed : False --------------------------------------------- INFO
hi!!!



## Append

`append` argument allows you to use `a+` or `w+` (default mode) writing mode. As instance:

In [19]:
write_results(fav_result, dirpath="out_files/write_results", append=True)

In [20]:
for filename in sorted(os.listdir(dir_path)):
    file_path = dir_path + filename
   
    with open(file_path) as f:
        print("\033[1m" + file_path + "\033[0;0m", f.read(), sep='\n')

out_files/write_results/dev1.group_1
---- dummy_task ** changed : False --------------------------------------------- INFO
hi!!!


task_with_data******************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv task_with_data ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
Spaghetti monster
^^^^ END task_with_data ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

out_files/write_results/dev2.group_1
task****************************************************************************
* dev2.group_1 ** changed : False **********************************************
vvvv task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
What's happening?
---- task_without_exception ** changed : False --------------------------------- INFO
All is fine
^^^^ END task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


task_with_data***********************